In [1]:
from pymongo import MongoClient

from ajna_commons.flask.conf import DATABASE, MONGODB_URI
from ajna_commons.utils.images import get_imagens_recortadas
from collections import defaultdict, Counter

db = MongoClient(host=MONGODB_URI)[DATABASE]

In [5]:
ENCONTRADOS = {'metadata.carga.atracacao.escala': {'$ne': None},
               'metadata.contentType': 'image/jpeg'}
               

def get_cursor_filtrdo(db, vazio=False, limit=None):
    filtro = ENCONTRADOS
    filtro['metadata.carga.vazio'] = vazio
    print(filtro)
    cursor = db['fs.files'].find(filtro)
    if limit:
        cursor.limit(limit)
    return cursor


In [53]:
def monta_dict_imagens_portipo(db, vazio=False, limit=None,
                               limitportipo=200):
    cursor = get_cursor_filtrado(db, vazio=vazio, limit=limit)
    imagens = {}
    imagens['40'] = defaultdict(list)
    imagens['20'] = defaultdict(list)
    tipo_counter = Counter()
    if vazio:
        CAPACIDADE = 'capacidade(pes)'
        TIPO = 'nometipo'
    else:
        CAPACIDADE = 'capacidade'
        TIPO = 'tipo'
    for linha in cursor:
        _id= linha['_id']
        containeres = linha.get('metadata').get('carga').get('container')
        for container in containeres:
            capacidade = container.get(CAPACIDADE)
            tipo = container.get(TIPO)
            if tipo is None:
                continue
            if tipo_counter[tipo] < limitportipo:
                lista = [(_id, im) for im in get_imagens_recortadas(db, _id)]
                if lista:
                    imagens[capacidade][tipo].append(lista)
                    tipo_counter[tipo] += 1
    return imagens

In [54]:
imagens = monta_dict_imagens_portipo(db, limit=50000)

{'metadata.carga.atracacao.escala': {'$ne': None}, 'metadata.carga.vazio': False, 'metadata.contentType': 'image/jpeg'}


In [55]:
imagensv = monta_dict_imagens_portipo(db, vazio=True, limit=20000)

{'metadata.carga.atracacao.escala': {'$ne': None}, 'metadata.carga.vazio': True, 'metadata.contentType': 'image/jpeg'}


In [56]:
import unicodedata
import string

valid_filename_chars = "-_.() %s%s" % (string.ascii_letters, string.digits)

def clean_filename(filename, whitelist=valid_filename_chars, replace=' '):
    # replace spaces
    for r in replace:
        filename = filename.replace(r,'_')
    
    # keep only valid ascii chars
    cleaned_filename = unicodedata.normalize('NFKD', filename).encode('ASCII', 'ignore').decode()
    
    # keep only whitelisted chars
    return ''.join(c for c in cleaned_filename if c in whitelist)

In [57]:
import os
from PIL import Image
def save_dicts(path, imagens):
    try:
        os.mkdir(path)
    except FileExistsError:
        pass
    for cap in ['20', '40']:
        cap_path = os.path.join(path, cap)
        try:
            os.mkdir(cap_path)
        except FileExistsError:
            pass
        for key, value in imagens[cap].items():
            sub_path = os.path.join(cap_path, clean_filename(key))
            try:
                os.mkdir(sub_path)
            except FileExistsError:
                pass
            for items in value:
                for imtuple in items:
                    image = imtuple[1]
                    filename = str(imtuple[0]) + '.jpg'
                    image.save(os.path.join(sub_path, filename))

In [58]:
save_dicts(path='naovazios', imagens=imagens)
save_dicts(path='vazios', imagens=imagensv)